<a href="https://colab.research.google.com/github/NaotoMano1031/NaotoMano1031/blob/main/DataProcessingGRF_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GAIT_Sample作成のための床反力データの抽出

事前準備:フォルダに必要なファイルが全て入っているようにする

参考サイト:https://note.nkmk.me/



Step 1 ファイルの抽出


In [4]:
import glob
import os
import pandas as pd
import numpy as np


#読み込みたいフォルダのディレクトリを指定
excel_files = glob.glob('/content/*.xlsx')
print(excel_files)

#読み込みたいシート名を選択
for b in excel_files:
    excel_df = pd.read_excel(b,sheet_name='GSD')#明示的に
    excel_df.to_csv(f'{b}.csv',header=False) 

    
files = glob.glob('/content/*.csv')
print(files)

['/content/2-8.xlsx', '/content/2-7.xlsx', '/content/2-2.xlsx', '/content/2-6.xlsx']
['/content/2-7.xlsx.csv', '/content/2-8.xlsx.csv', '/content/2-2.xlsx.csv', '/content/2-6.xlsx.csv']


Step 2 必要な値の計算と抽出

In [5]:
#空のリストを準備
data_list = ['id']
left_list = []
right_list = []

for a in files:
    df = pd.read_csv(a)
    data_list.append(os.path.basename(a).split('.',1)[0]) 
    df['filename'] = os.path.basename(a).split('.',1)[0]
    
    #Braking and AP impulse算出用(立脚期全体を抽出)
    #1行目が揃っていないデータがあればエラーが出ます
    left = pd.DataFrame(df.loc[0:100, ['nt(%)','左Fz(Ave)','左Fx(Ave)','filename']])\
    .astype({'nt(%)':float,'左Fz(Ave)':float,'左Fx(Ave)':float,})\
    .rename(columns= {'nt(%)': 'GC','左Fz(Ave)':'LtFz','左Fx(Ave)':'LtFx'})

    right = pd.DataFrame(df.loc[103:203, ['nt(%)','右Fz(Ave)','右Fx(Ave)','filename']])\
    .astype({'nt(%)':float,'右Fz(Ave)':float,'右Fx(Ave)':float,})\
    .rename(columns= {'nt(%)': 'GC','右Fz(Ave)':'RtFz','右Fx(Ave)':'RtFx'})
    
    left = left.query('-1 < GC < 101 ')
    right = right.query('-1 < GC < 101 ')
    
    #Mean,Impulse,Peak Propulsion算出用(立脚期0-20%を除去)
    left2 = left.query('19 < GC < 101 ').rename(columns= {'LtFz':'LtFz2','LtFx':'LtFx2'})
    right2 = right.query('19 < GC < 101 ').rename(columns= {'RtFz':'RtFz2','RtFx':'RtFx2'})

    #Mean Fz,Peak Braking,Fzを算出
    left3 = left.agg(['max','sum','mean','min']).drop('GC',axis = 1)
    right3 = right.agg(['max','sum','mean','min']).drop('GC',axis = 1)

    #AP-impulse,Mean Peak propulsion,PPratioを算出
    left4 = left2.query(' 0 < LtFx2').agg(['max','sum','mean']).drop(['GC','LtFz2','filename'],axis = 1)
    right4 = right2.query(' 0 < RtFx2').agg(['max','sum','mean']).drop(['GC','RtFz2','filename'],axis = 1)
      
    left5 = pd.concat([left3,left4],axis = 1)
    right5 = pd.concat([right3,right4],axis = 1)

    left_list.append(left5)
    right_list.append(right5)
    print(data_list)

    

['id', '2-7']
['id', '2-7', '2-8']
['id', '2-7', '2-8', '2-2']
['id', '2-7', '2-8', '2-2', '2-6']


Step 3 抽出されたデータを全てまとめ、書き出し

In [7]:
#今までのデータをまとめる
left_df = pd.concat(left_list,axis = 1).sort_index(axis = 1)
right_df = pd.concat(right_list, axis=1).sort_index(axis = 1)

name = pd.DataFrame(data_list)

#ここから書き出し
#麻痺側の判別はしていないためPareticPropulsionは別に算出が必要
import pathlib
pathlib.Path('Rt_result').mkdir()
os.chdir('Rt_result')
right_df.loc[['min'],['RtFx','filename']].to_csv('PeakBraking.csv')
right_df.loc[['max'],['RtFz']].to_csv('PeakFz.csv')
right_df.loc[['mean'],['RtFz']].to_csv('MeanFz.csv')
right_df.loc[['sum'],['RtFx']].to_csv('APimpulse.csv')
right_df.loc[['max'],['RtFx2']].to_csv('PeakPropulsion.csv')
right_df.loc[['mean'],['RtFx2']].to_csv('MeanPropulsion.csv')
right_df.loc[['sum'],['RtFx2']].to_csv('ImpulsePropulsion.csv')
name.to_csv("id.csv")

pathlib.Path('Lt_result').mkdir()
os.chdir('Lt_result')
left_df.loc[['min'],['LtFx','filename']].to_csv('PeakBraking.csv')
left_df.loc[['max'],['LtFz']].to_csv('PeakFz.csv')
left_df.loc[['mean'],['LtFz']].to_csv('MeanFz.csv')
left_df.loc[['sum'],['LtFx']].to_csv('APimpulse.csv')
left_df.loc[['max'],['LtFx2']].to_csv('PeakPropulsion.csv')
left_df.loc[['mean'],['LtFx2']].to_csv('MeanPropulsion.csv')
left_df.loc[['mean'],['LtFx2']].to_csv('ImpulsePropulsion.csv')
name.to_csv("id.csv")